In [15]:
import pandas as pd
import numpy as np
from unidecode import unidecode

In [16]:
data = pd.read_csv('../Data/data.csv', index_col=0)

In [17]:
data

,sex,surname,firstname,age,link,occupation
0,femme,Chardon,Marie,30.0,fille,idem
1,homme,Lhopital,Louis-Jean,67.0,chef,sp
2,femme,Papin,Marie,15.0,idem,idem
3,femme,Lavocat,Marie,NaN,femme,NaN
4,femme,Benne,Marguerite,78.0,NaN,NaN
...,...,...,...,...,...,...
236,femme,Burlurut,Pétronille,NaN,épouse,sans profession
237,homme,Combey,Alexandre,39.0,son fils,idem
238,femme,Collin,Marguerite,38.0,épouse,idem
239,homme,Dumont,Etienne,NaN,fils,NaN


In [24]:
# Remplacement des caractères spéciaux
# Homogénéisation du texte : suppression des majuscules
# "Homogénéisation" des valeurs manquantes/labels signifiants la même chose (sans prof, sp..) pour créer moins de tokens doublons. 
for column in ['link', 'age', 'occupation']:
    data[column] = data[column].replace(['néant', 'idem'], np.nan)
    
data['occupation'] = data['occupation'].replace(['s p', 'sans p.', 'sans p', 'sans prof', 'sans profession.', 's.p', 'sans profession', 's.p.', 'sans prof.', 'sp.'], 'sp')
def replace_special_characters(text):
    text = unidecode(text)
    return text

def homogene_link(x):
    if type(x) == str:
        if "fils" in x:
            return 'fils'
        elif "fille" in x:
            return "fille"
        elif 'enf' in x:
            return 'enfant'
        elif 'femme' in x:
            return 'femme'
        elif 'homme' in x:
            return 'homme'
        elif 'chef' in x or 'chf' in x:
            return 'chef'

data['link'] = data['link'].apply(homogene_link)

def lower_text(x):
    return x.lower()

for col in ['surname', 'firstname', 'link', 'occupation']:
    data[col] = data[col].astype(str)
    data[col] = data[col].apply(lower_text)
    data[col] = data[col].apply(replace_special_characters)
    data[col] = data[col].replace('nan', np.nan)
    data[col] = data[col].replace('none', np.nan)

data = data.dropna(subset=['firstname'])
data = data.reset_index(drop=True)

In [19]:
def labelization(row):
    if row == 'femme':
        return 0
    else:
        return 1

data['sex'] = data['sex'].apply(labelization)

In [31]:
data

,sex,surname,firstname,age,link,occupation,sentence
0,0,chardon,marie,30.0,fille,NaN,chardon marie 30.0 fille nan
1,1,lhopital,louis-jean,67.0,chef,sp,lhopital louis-jean 67.0 chef sp
2,0,papin,marie,15.0,NaN,NaN,papin marie 15.0 nan nan
3,0,lavocat,marie,NaN,femme,NaN,lavocat marie nan femme nan
4,0,benne,marguerite,78.0,NaN,NaN,benne marguerite 78.0 nan nan
...,...,...,...,...,...,...,...
235,0,burlurut,petronille,NaN,NaN,sp,burlurut petronille nan nan sp
236,1,combey,alexandre,39.0,fils,NaN,combey alexandre 39.0 fils nan
237,0,collin,marguerite,38.0,NaN,NaN,collin marguerite 38.0 nan nan
238,1,dumont,etienne,NaN,fils,NaN,dumont etienne nan fils nan


In [32]:
def sentence_data(row):
    # Concaténer les données textuelles en une seule chaîne
    text = ' '.join([str(row['surname']), str(row['firstname']), str(row['age']), str(row['link']), str(row['occupation'])])
    
    return text
data['sentence'] = data.apply(sentence_data, axis=1)

In [34]:
data.to_csv('../Data/data.csv')